In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import pandas as pd
import os   

# from data import OralCancerDataset, RotationTransform
# from utils import save_checkpoint, load_checkpoint, latest_checkpoint_path
# from process import train_epoch, validate_epoch, infer
# from models import *

In [3]:
batch_size = 224
num_epochs = 1000
learning_rate = 2e-05 #3e-5 10-  5e-5 reg
#min_learning_rate = 5e-06
betas = (0.9, 0.999)
num_workers = 4
weight_decay = 0 #2e-05
pin_memory = True 
drop_path_rate = 0.3
gamma = 0.7

save_freq = 10

path_to_csv = 'cancer-classification-challenge-2024/train.csv'
path_to_train_images = 'cancer-classification-challenge-2024/train'
path_to_test_images = 'cancer-classification-challenge-2024/test'
model_dir = 'checkpoints/'

transform = transforms.Compose([
    RotationTransform(angles=[0, -90, 90, 180]),
    transforms.Resize((128, 128)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
train_dataset = OralCancerDataset(path_to_csv=path_to_csv, path_to_image=path_to_train_images, train=1, transform=transform, val_split=0.5)
val_dataset = OralCancerDataset(path_to_csv=path_to_csv, path_to_image=path_to_train_images, train=0, transform=transform_val, val_split=0.5)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_convnext_tiny_AvgPolling(drop_path_rate=drop_path_rate).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,start_factor=1,end_factor=0.1,total_iters=20)
writer = SummaryWriter() 

In [4]:
current_min_val_loss = 1000
current_max_val_auc = 0

try:
    _, _, _, start_epoch = load_checkpoint(latest_checkpoint_path(model_dir, "checkpoint_*.pth"), model, optimizer)
    current_min_val_loss,_ , current_max_val_auc = validate_epoch(model, val_loader, loss_fn, start_epoch, writer, device)
    start_epoch += 1
    print(f"Starting from epoch {start_epoch}, min loss: {current_min_val_loss}, max auc: {current_max_val_auc}")
except:
    start_epoch = 0


for epoch in range(start_epoch, num_epochs + 1):
    
    train_loss, trian_acc, lr = train_epoch(model, optimizer, scheduler, loss_fn, train_loader, device)
    val_loss, val_acc, val_auc = validate_epoch(model, val_loader, loss_fn, epoch, writer, device)

    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar("HyperParameters/LearningRate", lr , epoch)
    writer.add_scalar("Accuracy/val", val_acc, epoch)
    writer.add_scalar("Accuracy/train", trian_acc, epoch)
    writer.add_scalar("AUC/val", val_auc, epoch)
    print(f"Epoch: {epoch:3d}/{num_epochs:3d} Train loss: {train_loss:6f}, Val loss: {val_loss:6f}, Train acc: {trian_acc:6f}, Val acc: {val_acc:6f}, Val AUC: {val_auc:6f}, LR: {lr:6f}")

    if epoch > 7 and val_loss < current_min_val_loss:
        current_min_val_loss = val_loss
        save_checkpoint(model, optimizer, scheduler, epoch, model_dir, max_checkpoints = 10)
        print(f"Model saved at epoch {epoch}, val loss: {val_loss}")
    elif epoch > 7 and val_auc > current_max_val_auc:
        current_max_val_auc = val_auc
        save_checkpoint(model, optimizer, scheduler, epoch, model_dir, max_checkpoints = 10)
        print(f"Model saved at epoch {epoch}, val auc: {val_auc}")
    elif epoch > 1 and epoch%save_freq == 0:
        save_checkpoint(model, optimizer, scheduler, epoch, model_dir, max_checkpoints = 10)
        print(f"Model saved at epoch {epoch}")

Epoch:   0/220 Train loss: 0.493014, Val loss: 0.403734, Train acc: 0.753466, Val acc: 80.967039, Val AUC: 0.874111, LR: 0.000019
Epoch:   1/220 Train loss: 0.375581, Val loss: 0.350352, Train acc: 0.825871, Val acc: 83.830019, Val AUC: 0.908684, LR: 0.000018
Epoch:   2/220 Train loss: 0.320417, Val loss: 0.305062, Train acc: 0.855646, Val acc: 86.120948, Val AUC: 0.927207, LR: 0.000017
Epoch:   3/220 Train loss: 0.291243, Val loss: 0.284201, Train acc: 0.870804, Val acc: 87.324980, Val AUC: 0.937182, LR: 0.000016
Epoch:   4/220 Train loss: 0.263542, Val loss: 0.271068, Train acc: 0.883870, Val acc: 87.771724, Val AUC: 0.946319, LR: 0.000015
Epoch:   5/220 Train loss: 0.246503, Val loss: 0.260856, Train acc: 0.892129, Val acc: 88.311087, Val AUC: 0.950406, LR: 0.000015
Epoch:   6/220 Train loss: 0.231341, Val loss: 0.251201, Train acc: 0.900369, Val acc: 88.926723, Val AUC: 0.954497, LR: 0.000014
Epoch:   7/220 Train loss: 0.215300, Val loss: 0.239858, Train acc: 0.908147, Val acc: 89.

KeyboardInterrupt: 

In [5]:
output_csv = 'outputs/submission_____.csv'

_, _, _, start_epoch = load_checkpoint(latest_checkpoint_path(model_dir, "checkpoint_12.pth"), model, optimizer)

image_files = [f for f in os.listdir(path_to_test_images) if f.endswith('.jpg')]
results = []

model.eval()
with torch.no_grad():
    for i, image_file in enumerate(image_files):
        image_path = os.path.join(path_to_test_images, image_file)
        image = Image.open(image_path).convert('RGB')
        image = transform_val(image).unsqueeze(0).to(device)
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        prob_class_1 = probabilities[0][1].item()  
        if (i % 100 == 0):
            print(f'Processed {i}/{len(image_files)} images')

        results.append((image_file, prob_class_1))

df = pd.DataFrame(results, columns=['Name', 'Diagnosis'])
df.to_csv(output_csv, index=False)

Loaded checkpoint 'checkpoints/checkpoint_12.pth' (epoch 12)
Processed 0/66530 images
Processed 100/66530 images
Processed 200/66530 images
Processed 300/66530 images
Processed 400/66530 images
Processed 500/66530 images
Processed 600/66530 images
Processed 700/66530 images
Processed 800/66530 images
Processed 900/66530 images
Processed 1000/66530 images
Processed 1100/66530 images
Processed 1200/66530 images
Processed 1300/66530 images
Processed 1400/66530 images
Processed 1500/66530 images
Processed 1600/66530 images
Processed 1700/66530 images
Processed 1800/66530 images
Processed 1900/66530 images
Processed 2000/66530 images
Processed 2100/66530 images
Processed 2200/66530 images
Processed 2300/66530 images
Processed 2400/66530 images
Processed 2500/66530 images
Processed 2600/66530 images
Processed 2700/66530 images
Processed 2800/66530 images
Processed 2900/66530 images
Processed 3000/66530 images
Processed 3100/66530 images
Processed 3200/66530 images
Processed 3300/66530 images

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os   

from data import OralCancerDataset, RotationTransform
from utils import save_checkpoint, load_checkpoint, latest_checkpoint_path
from process import train_epoch
from models import *
batch_size = 224
learning_rate = 1e-05
betas = (0.9, 0.999)
num_workers = 4
weight_decay = 3e-05
pin_memory = True
drop_path_rate = 0.625
gamma = 0.7


output_csv = 'outputs/submission_all_data_nxt_25_14__.csv'
path_to_csv = 'cancer-classification-challenge-2024/train.csv'
path_to_train_images = 'cancer-classification-challenge-2024/train'
path_to_test_images = 'cancer-classification-challenge-2024/test'
model_dir = 'checkpoints/'

transform = transforms.Compose([
    RotationTransform(angles=[0, -90, 90, 180]),
    transforms.Resize((128, 128)), 
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((128, 128)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = OralCancerDataset(path_to_csv=path_to_csv, path_to_image=path_to_train_images, transform=transform, full_dataset=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_convnext_tiny_AvgPolling(drop_path_rate=drop_path_rate).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)


_, _, _, start_epoch = load_checkpoint(latest_checkpoint_path(model_dir, "checkpoint_11.pth"), model, optimizer)

train_loss, trian_acc, lr = train_epoch(model, optimizer, None, loss_fn, train_loader, device)
save_checkpoint(model, optimizer, None, 0, model_dir, max_checkpoints = 10)
    

image_files = [f for f in os.listdir(path_to_test_images) if f.endswith('.jpg')]
results = []

model.eval()
with torch.no_grad():
    for i, image_file in enumerate(image_files):
        image_path = os.path.join(path_to_test_images, image_file)
        image = Image.open(image_path).convert('RGB')
        image = transform_val(image).unsqueeze(0).to(device)
        outputs = model(image)
        probabilities = F.softmax(outputs, dim=1)
        prob_class_1 = probabilities[0][1].item()  
        if (i % 100 == 0):
            print(f'Processed {i}/{len(image_files)} images')

        results.append((image_file, prob_class_1))

df = pd.DataFrame(results, columns=['Name', 'Diagnosis'])
df.to_csv(output_csv, index=False)

Loaded checkpoint 'checkpoints/checkpoint_11.pth' (epoch 11)


: 